Goal: Investigate different ROI combination

ex: forehead and cheeks, jaw and mustache, ...

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [2]:
# -- LOAD A DATASET

dataset_name = 'lgi_ppgi'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

videos = constants.get_video_settings(dataset_name)
print(videos)

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

{'GYM': array([ 0,  4,  8, 12, 16, 20]), 'RESTING': array([ 1,  5,  9, 13, 17, 21]), 'ROTATION': array([ 2,  6, 10, 14, 18, 22]), 'TALK': array([ 3,  7, 11, 15, 19])}
0 D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cv_camera_sensor_stream_handler.avi
1 D:/datasets_rppg/lgi_ppgi\alex\alex_resting\cv_camera_sensor_stream_handler.avi
2 D:/datasets_rppg/lgi_ppgi\alex\alex_rotation\cv_camera_sensor_stream_handler.avi
3 D:/datasets_rppg/lgi_ppgi\alex\alex_talk\cv_camera_sensor_stream_handler.avi
4 D:/datasets_rppg/lgi_ppgi\angelo\angelo_gym\cv_camera_sensor_stream_handler.avi
5 D:/datasets_rppg/lgi_ppgi\angelo\angelo_resting\cv_camera_sensor_stream_handler.avi
6 D:/datasets_rppg/lgi_ppgi\angelo\angelo_rotation\cv_camera_sensor_stream_handler.avi
7 D:/datasets_rppg/lgi_ppgi\angelo\angelo_talk\cv_camera_sensor_stream_handler.avi
8 D:/datasets_rppg/lgi_ppgi\cpi\cpi_gym\cv_camera_sensor_stream_handler.avi
9 D:/datasets_rppg/lgi_ppgi\cpi\cpi_resting\cv_camera_sensor_stream_handler.avi
10 D:/datasets_rp

In [3]:
# -- PARAMETER SETTING

roi = 'mustache'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for lgi_ppgi is 40 
Testing ROI = mustache with minimum 2 landmarks


# ROI

In [4]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [5]:
from itertools import chain, combinations

def get_combinations(elements, min_len=2, max_len=3):
    combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len,max_len)))
    return [tuple(i) for i in combs]

roi_list = [ele for ele in list(rois.keys()) if ele != 'temple' and ele != 'nose']
roi_combinations = get_combinations(roi_list, min_len, min_len+1)
print(len(roi_combinations),roi_combinations)

# roi_combinations = [('forehead', 'mustache'), ('mustache', 'jaw'), ('mustache', 'cheeks')]
landmarks_dict = dict()
for roi_combination in roi_combinations:
    landmarks_dict[roi_combination] = [ldmk for roi in list(map(rois.get, roi_combination)) for ldmk in roi]
landmarks_list = list(landmarks_dict.values())
landmarks_keys = list(landmarks_dict.keys())
print(landmarks_dict)

6 [('forehead', 'cheeks'), ('forehead', 'jaw'), ('forehead', 'mustache'), ('cheeks', 'jaw'), ('cheeks', 'mustache'), ('jaw', 'mustache')]
{('forehead', 'cheeks'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek'], ('forehead', 'jaw'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_marionette_fold', 'right_marionette_fold', 'chin'], ('forehead', 'mustache'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_nasolabial_fold', 'right_nasolabial_fold', 'left_upper_lip', 'right_upper_lip', 'philtrum'], ('cheeks', 'jaw'): ['left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'left_marionette_fold', 'right_marionette_fold', 'chin'], ('cheeks', 'mustache'): ['left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'left_nasolabial

# Pipeline

In [6]:
class CombineLandmarks:
    def __init__(self, videos, dataset, winsize, patch_size, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.losses = []
        self.load_ground_truth(videos)

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def process(self,landmarks):
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            print(videoName)
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=landmarks, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        print(f"Total loss for {landmarks}: {sum(losses)}")
        self.losses.append(losses)

    def fit(self, landmarks_list):
        for landmarks in landmarks_list:
            print("Processing landmarks: ", landmarks)
            self.process(landmarks)

In [25]:
idx = 3
print(landmarks_keys[idx], landmarks_dict[landmarks_keys[idx]])


('cheeks', 'jaw') ['left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'left_marionette_fold', 'right_marionette_fold', 'chin']


In [26]:
pl = custom_pipeline.LandmarksPipeline()
# np.arange(0, len(dataset.videoFilenames))
model = CombineLandmarks(np.arange(0, len(dataset.videoFilenames)), dataset, wsize, patch_size, pl, methods=['cupy_CHROM'], verb=False)

idx = 5
print(landmarks_keys[idx], landmarks_dict[landmarks_keys[idx]])
model.fit([landmarks_dict[landmarks_keys[idx]]])

4: Unusable data.
16: Unusable data.
Video name:  {22} ['D:/datasets_rppg/lgi_ppgi\\alex\\alex_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\alex\\alex_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\angelo\\angelo_talk\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_gym\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_resting\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_rotation\\cv_camera_sensor_stream_handler.avi', 'D:/datasets_rppg/lgi_ppgi\\cpi\\cpi_talk\\cv_camera_sensor_stream_handler.avi', 'D:/data

In [ ]:
df = model.res.reset_index(drop=True)
df.head(1)

,method,dataset,videoIdx,sigFilename,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,bpmGT,bpmES,bpmES_mad,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,,,,D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cv_cam...,[38.36790400628634],[30.23488331330128],[0.279113156416931],[0.15404849139646531],[-6.257202795849969],[87.462890625],"[16.4794921875, 4.39453125, 1.46484375, 1.4648...","[101.0, 100.0, 97.0, 94.5, 93.0, 93.0, 93.0, 9...","[74.3408203125, 54.931640625, 50.537109375, 47...",,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",188.866689,"[left_marionette_fold, right_marionette_fold, ..."


In [ ]:
print(roi_combinations[idx])
df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
df['videoIdx'] = df['videoFilename'].apply(lambda x: allvideo.index(x))
if dataset_name == 'lgi_ppgi':
  df['videoFilename'] = df['videoFilename'].apply(lambda x: x.split('\\')[2])
if dataset_name == 'mr_nirp':
  df['videoFilename'] = df['videoFilename'].apply(lambda x: x.split('\\')[1])
df['dataset'] = f'{dataset_name}'
df['roi'] = '_'.join(roi_combinations[idx])
# df['dataset'] = f'{dataset_name}_{roi}'
# if dataset_name == 'lgi_ppgi':
#   df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
# if dataset_name == 'mr_nirp':
#   df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['sigFilename', 'bpmES_mad', 'videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

from fastdtw import fastdtw
df.insert(11, 'DTW', None)
for row in df.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df.loc[row.Index, 'DTW'] = [distance]

print(df.shape, df.landmarks.unique().size)
df.head(1)

('jaw', 'mustache')
(22, 19) 1


,method,dataset,videoFilename,videoIdx,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,DTW,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks,roi
0,cupy_CHROM,lgi_ppgi,alex_gym,0,[38.36790400628634],[30.23488331330128],[0.279113156416931],[0.15404849139646531],[-6.257202795849969],[87.462890625],"[16.4794921875, 4.39453125, 1.46484375, 1.4648...",[7354.14453125],"[101.0, 100.0, 97.0, 94.5, 93.0, 93.0, 93.0, 9...","[74.3408203125, 54.931640625, 50.537109375, 47...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",188.866689,"(left_marionette_fold, right_marionette_fold, ...",jaw_mustache


In [24]:
old = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5')
new = pd.concat([old, df])
new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5', key='df', mode='w')

C:\Users\erolland\AppData\Local\Temp\3\ipykernel_15624\221195730.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'MAD', 'DTW', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks', 'roi'],
      dtype='object')]

  new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5', key='df', mode='w')


In [35]:
x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5', key='df',)
print(x.shape, x.landmarks.unique().size, x.dataset.unique().size )
print(x.dataset.unique())

(132, 18) 6 6
['lgi_ppgi_forehead_cheeks' 'lgi_ppgi_forehead_jaw' 'lgi_ppgi_jaw_cheeks'
 'lgi_ppgi_forehead_mustache' 'lgi_ppgi_cheeks_mustache'
 'lgi_ppgi_jaw_mustache']


In [127]:
# pd.DataFrame.from_dict(landmarks_dict, orient='index',)
from itertools import chain, combinations

def get_combinations(elements, min_len=2, max_len=3):
    combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len,max_len)))
    return [tuple(i) for i in combs]

roi_list = [ele for ele in list(rois.keys()) if ele != 'temple' and ele != 'nose']
roi_combinations = get_combinations(roi_list, min_len, min_len+1)
print(len(roi_combinations),roi_combinations)

# roi_combinations = [('forehead', 'mustache'), ('mustache', 'jaw'), ('mustache', 'cheeks')]
landmarks_dict = dict()
for roi_combination in roi_combinations:
    landmarks_dict[roi_combination] = [ldmk for roi in list(map(rois.get, roi_combination)) for ldmk in roi]
landmarks_list = list(landmarks_dict.values())
print(landmarks_dict)

print(landmarks_dict)
df_ldmk = pd.DataFrame([landmarks_dict.keys(), landmarks_dict.values()], index=['landmarks', 'landmarks_list']).T
df_ldmk['landmarks_list'] = df_ldmk['landmarks_list'].apply(lambda x: tuple(x))
df_ldmk['landmarks'] = df_ldmk['landmarks'].apply(lambda x: f'{dataset_name}_'+'_'.join(x))
df_ldmk = df_ldmk.rename({'landmarks':'dataset','landmarks_list':'landmarks'}, axis=1)
df_ldmk
# df.merge(df_ldmk, on='landmarks', how='left')

6 [('forehead', 'cheeks'), ('forehead', 'jaw'), ('forehead', 'mustache'), ('cheeks', 'jaw'), ('cheeks', 'mustache'), ('jaw', 'mustache')]
{('forehead', 'cheeks'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek'], ('forehead', 'jaw'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_marionette_fold', 'right_marionette_fold', 'chin'], ('forehead', 'mustache'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_nasolabial_fold', 'right_nasolabial_fold', 'left_upper_lip', 'right_upper_lip', 'philtrum'], ('jaw', 'cheeks'): ['left_marionette_fold', 'right_marionette_fold', 'chin', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek'], ('cheeks', 'mustache'): ['left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'left_nasolabial

,dataset,landmarks
0,lgi_ppgi_forehead_cheeks,"(lower_medial_forehead, glabella, left_lower_l..."
1,lgi_ppgi_forehead_jaw,"(lower_medial_forehead, glabella, left_lower_l..."
2,lgi_ppgi_forehead_mustache,"(lower_medial_forehead, glabella, left_lower_l..."
3,lgi_ppgi_jaw_cheeks,"(left_marionette_fold, right_marionette_fold, ..."
4,lgi_ppgi_cheeks_mustache,"(left_malar, right_malar, left_lower_cheek, ri..."
5,lgi_ppgi_jaw_mustache,"(left_marionette_fold, right_marionette_fold, ..."


In [130]:
# n = new.merge(df_ldmk, on='landmarks', how='left')
# n['dataset_x'] = n['dataset_y']
# # n.dataset_x.unique()
# n = n.rename({'dataset_x':'dataset'}, axis=1).drop(columns=['dataset_y'])
# n.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5', key='df')

C:\Users\erolland\AppData\Local\Temp\3\ipykernel_26220\3922332175.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'MAD', 'DTW', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks'],
      dtype='object')]

  n.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_combine_roi.h5', key='df')
